# NIDS based on Network Flow Measurements

### What is network flow and flow measurement?

A network flow consists of all network packets that have the same flow identifier (ID). The flow ID can be extracted from the packet header and is usually defined by the 5-tuple: source and destination IP addresses, source and destination ports, and the protocol that is used at the transport layer.

<center>
<img src="images/counting/5tuple.png"/>
</center>

Flow measurement tries to gather metadata on existing flows like the presence of certain features, the average volume, or peak volumes. Measuring the flow data is indispensable in a number of applications such as traffic analysis, network visibility, congestion control, heavy-hitter detection, anomaly detection, and intrusion detection.

Flow-measurement based NIDS use the network flow data (flow size, flow volume, flow features, etc.) for intrusion detection.

### Getting started

For network flow measurement on high-speed networks, especially when the focus is on hardware, we need to prioritize two things to enable processing at line-rate: high-speed lookup and high-speed counting. Keeping the memory and storage requirement within the on-chip memory is the best way to reduce the latency in lookup and counting, and this can be done by using efficient probabilistic lookup data structures and approximate counting techniques. In this tutorial, we focus only on high-speed lookup. However, a useful introduction to approximate counting is given in the closing notebook.

In this exercise we take flow size/flow volume as our parameter and we consider measuring/counting the flow size/flow volume of flow IDs to detect an anomaly and intrusion. We define **flow size** as the number of packets and **flow volume** as the byte volume of packets. We start with flow size first and later with flow volume.

This exercise will walk you through the simplest of data structures and will illustrate how malicious flows can be detected through flow measurements.

Similar to the earlier notebooks, don't forget to preload the dataset first !!

In [1]:
from lib.dataset import NIDSDataset

data_file = 'data/dataset_packets_v2.npy'
labels_file = 'data/dataset_labels_v1.npy'

dataset = NIDSDataset(data_file, labels_file)

We need to keep count for every flow ID that is passing. Therefore an array will be used that stores all the count values. In Python, we can make use of a dictionary (a key-value pair data structure) to store the flow IDs and their corresponding flow sizes. 

In [2]:
# INITIALISE THE COUNTERS TO ZERO

wordcounter=0
flowid = ""

# Python dictionary to store the flowid and its sizes. 
# Here we are taking only flow sizes 
# (i.e; the size of  each packet is taken as 1)
library = {} 

# loop over all datasets
for d in dataset:
    
    wordcounter = 0
    flowid = ""
    flowid_complete = 0
    # loop over all words
    for word in d:

        # examine if Ethertype is 0x0800 - in link layer header
        # if Ethertype is not equal to 0x0800, break
        """WRITE code here"""
        if wordcounter == 3:
            if(word[0] == 8) and (word[1] == 0):
                decision_is_made = 0
            else:
                decision_is_made = 1
                break
        
        # examine if proto is tcp or udp 6/17 - in network layer header
        # if proto is not tcp or udp, break
        """WRITE code here"""
        if wordcounter == 5:
            if(word[3] == 6)or(word[3] == 17):
                decision_is_made = 0
            else:
                decision_is_made = 1
                break
                
        # Extract flowid. Flowid is source IPv4 address, dest IPv4 address, 
        # source port, dest port
        # hint: convert numbers to hex and remove the leading 0x. 
        #       then concatenate together
        
        # extract Source Address - in network layer header
        """WRITE code here"""
        if wordcounter == 6:
            flowid += hex(word[2])[2:]  # ip SA 1/4
            flowid += hex(word[3])[2:]  # ip SA 2/4
        if wordcounter == 7:
            flowid += hex(word[0])[2:]  # ip SA 3/4
            flowid += hex(word[1])[2:]  # ip SA 4/4
        
        # extract Destination Address - in network layer header
        """WRITE code here"""
        if wordcounter == 7:
            flowid += hex(word[2])[2:]  # ip DA 1/4
            flowid += hex(word[3])[2:]  # ip DA 2/4
        if wordcounter == 8:
            flowid += hex(word[0])[2:]  # ip DA 3/4
            flowid += hex(word[1])[2:]  # ip DA 4/4
        
        # extract source port estination port - in network layer header
        """WRITE code here"""
        if wordcounter == 8:
            flowid += hex(word[2])[2:]  # ip SPort 1/2
            flowid += hex(word[3])[2:]  # ip SPort 2/2

        # examine Destination port - in transport layer header
        # set the flag flowid_complete to 1 when the flowid is complete, and break the loop
        # If the flowid is complete, update the dictionary
        """WRITE code here"""
        if wordcounter == 9:
            flowid += hex(word[0])[2:]  # ip DPort 1/2
            flowid += hex(word[1])[2:]  # ip DPort 2/2
            flowid_complete = 1
            break
        
        wordcounter += 1
    
    # Check if the extracted flowid is present in library. if not
    # add the flowid with a size 1. If present, increment the size by 1.
    if(flowid_complete==1):
        """WRITE code here"""
        if flowid in library:
            library[flowid] += 1
        else:
            library[flowid] = 1
            
    # end of iteration over words

# PRINTS all the flowids along with the sizes
for flowid in library:  
    print(flowid, '->', library[flowid])


c0a8a32c0a8a3db2ccc4 -> 32
c0a8a3c0a8a32cc4db2c -> 16
c0a8a32c0a8a3a4a0185 -> 32
c0a8a3c0a8a32185a4a0 -> 16
c0a8a9e000fcf6ea14eb -> 22
c0a8a9c0a8a347058 -> 7
c0a8a3c0a8a905847 -> 8
c0a8a9c0a8a348058 -> 9
c0a8a3c0a8a905848 -> 8
c0a8a9c0a8a349058 -> 9
c0a8a3c0a8a905849 -> 10
c0a8a9c0a8a34a058 -> 7
c0a8a3c0a8a90584a -> 8
c0a8a9c0a8a34c058 -> 9
c0a8a3c0a8a90584c -> 10
c0a8a9c0a8a34b185 -> 13
c0a8a3c0a8a91854b -> 13
c0a8a9c0a8a344087 -> 11
c0a8a3c0a8a908744 -> 6
c0a8a9c0a8a345c27 -> 21
c0a8a3c0a8a9c2745 -> 14
c0a8a9c0a8a3089089 -> 20
c0a8a3c0a8a9089089 -> 20
c0a8a9c0a8a3c115035 -> 2
c0a8a3c0a8a9035c115 -> 2
c0a8a9c0a8a3ddc035 -> 2
c0a8a3c0a8a9035ddc -> 2
c0a8a9c0a8a3ffe8035 -> 2
c0a8a3c0a8a9035ffe8 -> 2
c0a8a9c0a8a3f4a1035 -> 2
c0a8a3c0a8a9035f4a1 -> 2
c0a8a9c0a8a3461bd -> 25
c0a8a3c0a8a91bd46 -> 20
c0a8a9c0a8a3dde185 -> 2
c0a8a3c0a8a9185dde -> 2
c0a8a9c0a8a3ddf185 -> 2
c0a8a3c0a8a9185ddf -> 2
c0a8a9c0a8a3fc13035 -> 2
c0a8a3c0a8a9035fc13 -> 2
c0a8a9c0a8a3f4a2185 -> 2
c0a8a3c0a8a9185f4a2 -> 

To determine a possible malicious flow, a maximum can be put on the flow size. Setting the exact number of the threshold is a delicate task and requires experience.

In the next exercise, we set a threshold of **20** to determine which of the flowids exhibits anomalous behaviour by exceeding the allocated bandwidth.

In [3]:
"""Print the flow IDs that exceed a threshold"""

threshold = 20

# PRINTS all the flowids along with the sizes
for flowid in library:
    if(library[flowid]>threshold):
        print(flowid, '->', library[flowid])


c0a8a32c0a8a3db2ccc4 -> 32
c0a8a32c0a8a3a4a0185 -> 32
c0a8a9e000fcf6ea14eb -> 22
c0a8a9c0a8a345c27 -> 21
c0a8a9c0a8a3461bd -> 25
c0a8a19c0a8aff08a08a -> 48
c0a8a32c0a8aff08a08a -> 46
c0a8a19c0a8a3089089 -> 30
c0a8a3c0a8a19089089 -> 30
c0a8a19e000fb14e914e9 -> 28


<center><div style="background-color: #10FF107f;">The code above should report that there are 195 FlowIDs in the dataset, of which 10 exceed the allowed bandwidth.</div></center>

<hr/>
<center>
Continue with the <a href="21_counting.ipynb">next notebook</a> in a new browser tab.<br/><br/>
<img src="images/footer.png"/>
</center>